In [1]:
import Bio
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

In [2]:
nat_df = pd.read_excel('uniprot_reviewed_signalpeptides.xlsx')
gen_df = pd.read_excel('sp_prot_translations.xls')

In [3]:
def get_start(row):
    key = row['Signal peptide']
    parts = key.split()
    t = parts[1]
    try:
        return int(t)
    except:
        return None

def get_end(row):
    t = row['Signal peptide'].split()[2]
    try:
        return int(t)
    except:
        return None
    
def get_sp(row):
    seq = row['Sequence']
    
    start = row['start']
    end = row['end']
    
    try:
        return seq[int(start)-1:int(end)]
    except:
        return None

In [4]:
nat_df['start'] = nat_df.apply(lambda row: get_start(row), axis=1)
nat_df['end'] = nat_df.apply(lambda row: get_end(row), axis=1)
nat_df['sp'] = nat_df.apply(lambda row: get_sp(row), axis=1)
# nat_df.to_csv('nat_sps_parsed.csv')
nat_sps = list(set(nat_df['sp'].values))
len(nat_sps)

31475

In [5]:
gen_sps = []

gen_sps += [s[:-6] for s in gen_df['75'].values]
gen_sps += [s[:-6] for s in gen_df['90'].values]
gen_sps += [s[:-6] for s in gen_df['95'].values]
gen_sps += [s[:-6] for s in gen_df['99'].values]

len(gen_sps)

164

In [6]:
from Bio import pairwise2

In [ ]:
try:
    master_df = pd.read_csv('sp_top_100_scores.csv')
except:
    master_df = pd.DataFrame()

    for i, s1 in tqdm(enumerate(gen_sps)):
        alns = []
        for s2 in nat_sps:
            if s2:
                alignments = pairwise2.align.globalms(s1, s2, 2, -1, -1, -1)
                _, _, score, _, _ = alignments[0]
                alns.append([s1, s2, score])

        df = pd.DataFrame(alns, columns=['generated', 'natural', 'score'])
        df.sort_values(by='score', ascending=False, inplace=True)
        df = df.iloc[:100]
        master_df = pd.concat([master_df, df])

    master_df.to_csv('sp_top_100_scores.csv')

In [13]:
print('done')

done


In [14]:
master_df

,generated,natural,score
5605,MRFFGIHLALALATTSFA,MRFFETLALALLTTGALA,17.0
17972,MRFFGIHLALALATTSFA,MRQFTHGTLLAILALANTISA,17.0
26852,MRFFGIHLALALATTSFA,MKVFILALLALTATTAIA,16.0
26832,MRFFGIHLALALATTSFA,MQFTHLVALALALATSEA,16.0
7270,MRFFGIHLALALATTSFA,MKFLLVFALALATTSA,16.0
...,...,...,...
22239,MRSLLLTSALAALVSLAAASA,MPSRGCSCWLLSLALLCSLAAA,16.0
6666,MRSLLLTSALAALVSLAAASA,MIRKHSLGFVASALALAVSAQAFA,16.0
28126,MRSLLLTSALAALVSLAAASA,MRSTLLFVAILALSLAWSLG,16.0
267,MRSLLLTSALAALVSLAAASA,MRLAASSLLLGAAASLLSSATALA,16.0


In [ ]:
a = master_df[master_df['generated'] == gen_sps[0]]
a

In [ ]:
d

In [ ]:
# import pickle
# pickle.dump(alns, open('gen_nat_alns.p','wb'))

In [ ]:
# df = pd.DataFrame(alns, columns=['generated', 'natural', 'score'])
# df.to_csv('gen_gat_alns.csv')
# df